# 🚀 AION GPU Worker - Google Colab

Este notebook transforma seu Google Colab em um **GPU Worker gratuito** para o AION!

## ✨ O que este worker faz:
- **Inferência LLM**: Responde queries usando modelos customizados (Llama, Mistral, etc.)
- **Training LoRA**: Fine-tuna modelos com suas conversas de alta qualidade
- **Embeddings**: Gera embeddings localmente (elimina custo OpenAI)
- **Zero Custo**: Usa GPUs gratuitas do Google Colab!

## 📊 GPU Disponível:
- Tesla T4 (15GB VRAM) - mais comum
- ~12h/dia de uso gratuito
- Perfeito para fine-tuning e inferência!

## ⚙️ Configuração:
1. Substitua `AION_API_URL` pela URL do seu AION (Replit)
2. Execute todas as células
3. O worker se registrará automaticamente e começará a processar jobs!

---

In [ ]:
# ============================================================================
# CONFIGURAÇÃO - EDITE AQUI!
# ============================================================================

AION_API_URL = "https://seu-repl.replit.app"  # ⚠️ SUBSTITUA pela sua URL do Replit!
WORKER_NAME = "Colab-GPU-1"  # Nome único para este worker
GOOGLE_ACCOUNT_EMAIL = "sua-conta@gmail.com"  # Sua conta Google (para tracking)

# ============================================================================

In [ ]:
# 📦 Instalar dependências
!pip install -q transformers accelerate bitsandbytes peft torch requests GPUtil

print("✅ Dependências instaladas!")

In [ ]:
# 🔍 Detectar GPU disponível
import torch
import GPUtil

if torch.cuda.is_available():
    gpu = GPUtil.getGPUs()[0]
    print(f"✅ GPU Detectada: {gpu.name}")
    print(f"   VRAM Total: {gpu.memoryTotal}MB")
    print(f"   VRAM Livre: {gpu.memoryFree}MB")
    
    GPU_MODEL = gpu.name
    VRAM_GB = int(gpu.memoryTotal / 1024)
else:
    print("❌ ERRO: GPU não detectada!")
    print("⚠️  Vá em: Runtime > Change runtime type > Hardware accelerator > GPU")
    raise RuntimeError("GPU not available")

In [ ]:
# 🤖 Carregar modelo LLM (exemplo: Llama-3.2-1B)
# Você pode trocar por outros modelos depois!

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"  # Modelo pequeno para teste

print(f"📥 Carregando modelo: {MODEL_NAME}...")

# Configuração 4-bit para economizar VRAM
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

print("✅ Modelo carregado com sucesso!")

In [ ]:
# 🔗 Registrar worker no AION
import requests
import json

def register_worker():
    """Registra este worker no AION e recebe API key"""
    
    url = f"{AION_API_URL}/api/gpu/workers/register"
    
    payload = {
        "tenantId": 1,  # Default tenant
        "name": WORKER_NAME,
        "workerType": "colab",
        "gpuModel": GPU_MODEL,
        "vramGB": VRAM_GB,
        "capabilities": {
            "inference": True,
            "training": True,
            "embeddings": True,
            "maxBatchSize": 8,
            "supportedModels": [MODEL_NAME]
        },
        "metadata": {
            "accountEmail": GOOGLE_ACCOUNT_EMAIL,
            "region": "colab-us",
            "quotaHoursPerWeek": 84,  # ~12h/day * 7 days
            "usedHoursThisWeek": 0
        }
    }
    
    response = requests.post(url, json=payload)
    
    if response.status_code == 200:
        data = response.json()
        worker_id = data["worker"]["id"]
        api_key = data["worker"]["apiKey"]
        
        print(f"✅ Worker registrado com sucesso!")
        print(f"   Worker ID: {worker_id}")
        print(f"   API Key: {api_key[:20]}...")
        
        return api_key
    else:
        print(f"❌ Erro ao registrar worker: {response.status_code}")
        print(response.text)
        raise RuntimeError("Worker registration failed")

WORKER_API_KEY = register_worker()

In [ ]:
# 💓 Heartbeat - Mantém worker online
import time
from threading import Thread

def send_heartbeat():
    """Envia heartbeat a cada 30 segundos"""
    while True:
        try:
            url = f"{AION_API_URL}/api/gpu/workers/heartbeat"
            payload = {
                "apiKey": WORKER_API_KEY,
                "status": "online"
            }
            requests.post(url, json=payload, timeout=10)
        except Exception as e:
            print(f"⚠️  Heartbeat falhou: {e}")
        
        time.sleep(30)  # A cada 30 segundos

# Iniciar heartbeat em background
heartbeat_thread = Thread(target=send_heartbeat, daemon=True)
heartbeat_thread.start()

print("✅ Heartbeat ativo! Worker está online.")

In [ ]:
# 🎯 Processar jobs do AION
def generate_text(prompt, max_tokens=512, temperature=0.7):
    """Gera texto usando o modelo LLM"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text

def process_job(job):
    """Processa um job de inferência"""
    job_id = job["id"]
    job_type = job["jobType"]
    payload = job["payload"]
    
    print(f"🎯 Processando job {job_id} ({job_type})...")
    
    try:
        if job_type == "inference":
            prompt = payload.get("prompt", "")
            max_tokens = payload.get("maxTokens", 512)
            temperature = payload.get("temperature", 0.7)
            
            text = generate_text(prompt, max_tokens, temperature)
            tokens_generated = len(tokenizer.encode(text))
            
            result = {
                "text": text,
                "tokensGenerated": tokens_generated
            }
            
            # Reportar conclusão
            url = f"{AION_API_URL}/api/gpu/jobs/{job_id}/complete"
            headers = {"Authorization": f"Bearer {WORKER_API_KEY}"}
            requests.post(url, json={"result": result}, headers=headers)
            
            print(f"✅ Job {job_id} concluído! Gerados {tokens_generated} tokens.")
        else:
            print(f"⚠️  Job type '{job_type}' não suportado ainda.")
    
    except Exception as e:
        print(f"❌ Erro processando job {job_id}: {e}")
        
        # Reportar falha
        url = f"{AION_API_URL}/api/gpu/jobs/{job_id}/fail"
        headers = {"Authorization": f"Bearer {WORKER_API_KEY}"}
        requests.post(url, json={"error": str(e)}, headers=headers)

def worker_loop():
    """Loop principal: busca e processa jobs continuamente"""
    print("\n🚀 Worker iniciado! Aguardando jobs...\n")
    
    while True:
        try:
            # Buscar próximo job
            url = f"{AION_API_URL}/api/gpu/jobs/next"
            headers = {"Authorization": f"Bearer {WORKER_API_KEY}"}
            response = requests.get(url, headers=headers, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                job = data.get("job")
                
                if job:
                    process_job(job)
                else:
                    # Sem jobs, aguardar
                    time.sleep(5)
            else:
                print(f"⚠️  Erro buscando job: {response.status_code}")
                time.sleep(10)
        
        except KeyboardInterrupt:
            print("\n🛑 Worker interrompido pelo usuário.")
            break
        except Exception as e:
            print(f"⚠️  Erro no loop: {e}")
            time.sleep(10)

# Iniciar worker
worker_loop()

---

## 🎉 Parabéns!

Seu GPU Worker está rodando e processando jobs do AION!

### ✅ O que está acontecendo:
- Worker está **online** e visível no AION Dashboard
- Heartbeat enviado a cada 30 segundos
- Aguardando jobs de inferência/training
- Processamento **100% gratuito** com GPU do Colab!

### 📊 Monitorar:
Vá para o AION Dashboard > GPU Pool para ver:
- Status do worker (online/busy/offline)
- Jobs completados
- Tokens processados
- Latência média

### ⚠️ Importante:
- **Sessão máxima**: 12 horas (Colab desconecta automaticamente)
- **Idle timeout**: 90 minutos sem atividade
- **Reexecutar**: Se desconectar, reexecute todas as células!

---

**💡 Dica**: Abra múltiplos Colabs em diferentes contas Google para ter mais GPUs simultâneas!
